In [ ]:
%%bash
pip -q install trl
pip -q install peft
pip -q install torch
pip -q install datasets
pip -q install transformers
pip -q install peft

In [ ]:
import torch
from trl import SFTTrainer
from datasets import load_from_disk, load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
import pandas as pd

In [ ]:
# df = pd.read_csv('TRAIN_DATA.csv')
# df

# data_files = {"train":"TRAIN_DATA.parquet","test":"TEST_DATA.parquet"}
# raw_datasets = load_dataset("parquet", data_dir='./', data_files=data_files)
# raw_datasets

# train_dataset = load_from_disk('./TRAIN_DATA.parquet')

# from datasets import load_dataset
dataset = load_dataset('csv', data_files='TRAIN_DATA.csv')
dataset_A = dataset['train']

# dataset = load_dataset('csv', data_files={'train': 'TRAIN_DATA.csv', 'test': 'TRAIN_DATA.csv'})
dataset_A

In [ ]:
import textwrap

In [ ]:
for index in range(3):
    print("---"*15)
    print("Instruction: {}".format(textwrap.fill(df.iloc[index]["instruction"],width=50)))
    print("Output: {}".format(textwrap.fill(df.iloc[index]["output"],width=50)))
    print("Text: {}".format(textwrap.fill(df.iloc[index]["text"],width=50)))

In [ ]:
pretrained_model_name = "Salesforce/xgen-7b-8k-base"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name, torch_dtype=torch.bfloat16)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name, trust_remote_code=True)

In [ ]:
model_training_args = TrainingArguments(
    output_dir="xgen-7b-8k-base-fine-tuned",
    per_device_train_batch_size=4,
    optim="adamw_torch",
    logging_steps=80,
    learning_rate=2e-4,
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    num_train_epochs=100,
    save_strategy="epoch"
)

In [ ]:
lora_peft_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.05, task_type="CAUSAL_LM")

In [ ]:
SFT_trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_A,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=model_training_args,
    peft_config=lora_peft_config,
)